<a href="https://colab.research.google.com/github/IdaCy/polAItness_internals/blob/main/notebooks/modules_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Pipeline for Collab-Modules

Here are the steps we used to run the functions put into "collab_modules" for standardised setup.

In [1]:
# Check Python version (optional):
import sys
print("Python version:", sys.version)

# Get installations
!pip install --quiet torch numpy matplotlib scikit-learn pandas
!pip install --quiet huggingface_hub transformers

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# If you want to check GPU usage:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00
Using device: cuda


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# After running this cell, follow the link to grant Colab access to your Google Drive.

Mounted at /content/drive


In [3]:
!git clone https://github.com/IdaCy/polAItness_internals.git
%cd polAItness_internals

Cloning into 'polAItness_internals'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 181 (delta 78), reused 146 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (181/181), 242.43 KiB | 550.00 KiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/polAItness_internals


In [4]:
!pip install huggingface_hub --quiet

from huggingface_hub import notebook_login

# This will prompt you in Colab to enter your HF token or log in directly
notebook_login()

In [5]:
!git pull

Already up to date.


In [7]:
import logging
logging.basicConfig(level=logging.INFO)

from collab_modules.inference import load_model_and_tokenizer, run_inference

# 1. Load the model/tokenizer once
tokenizer, model = load_model_and_tokenizer(
    model_name="google/gemma-2-2b-it",
    use_bfloat16=True,
    hf_token=None
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [14]:
%ls prompts

nicer.csv  normal.csv  pure.csv  urgent.csv


In [9]:
from collab_modules.load_csv_prompts import load_prompts

# 2. Load one set of prompts
prompts_file_1 = "prompts/normal.csv"
prompts_jb = load_prompts(prompts_file_1, num_samples=10)

In [10]:
### just some logging ###

# Create (or get) a logger and set the level
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# OPTIONAL: if you want to clear any existing handlers, do:
# logger.handlers = []

# Create a console handler that prints to stdout
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)

# Create a formatter and set it for the console handler
formatter = logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s")
console_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(console_handler)


In [21]:
# 3. Run inference & capture activations
run_inference(
    model,
    tokenizer,
    prompts_jb,
    output_dir="output/extractions/gemma2-2b-it/normal",
    batch_size=8,
    max_new_tokens=50,
    extract_hidden_layers=[0,5,10,15,20,25],
    extract_attention_layers=[0,5,10,15,20,25],
    top_k_logits=10
)

== [Batch 0 of 2] Prompts 0-8 ==
=== Inference is complete! ===


In [23]:
# again: 2. Load one set of prompts - now urgent
prompts_file_1 = "prompts/nicer.csv"
prompts_jb = load_prompts(prompts_file_1, num_samples=10)

# again: 3. Run inference & capture activations - now nicer
run_inference(
    model,
    tokenizer,
    prompts_jb,
    output_dir="output/extractions/gemma2-2b-it/nicer",
    batch_size=8,
    max_new_tokens=50,
    extract_hidden_layers=[0,5,10,15,20,25],
    extract_attention_layers=[0,5,10,15,20,25],
    top_k_logits=10
)

== [Batch 0 of 2] Prompts 0-8 ==
=== Inference is complete! ===


In [25]:
%ls output/extractions/gemma2-2b-it/normal
%ls output/extractions/gemma2-2b-it/nicer

activations_00000_00008.pt  activations_00008_00010.pt
activations_00000_00008.pt  activations_00008_00010.pt


In [13]:
import sys
sys.path.append(".")

from collab_modules.read_predictions import read_predictions

# Then call it:
preds = read_predictions(
    read_output_dir="output/extractions/gemma2-2b-it/normal",
    max_predictions=50,
    write_predictions_file="logs/pred_out.txt",
    log_file="logs/readpred.log"
)

print("Number of preds:", len(preds))
print("First 3 preds:", preds[:15])

[INFO] === Starting read_predictions function ===
INFO:ReadPredictionsLogger:=== Starting read_predictions function ===
[INFO] read_output_dir = output/extractions/gemma2-2b-it/sample
INFO:ReadPredictionsLogger:read_output_dir = output/extractions/gemma2-2b-it/sample
[INFO] log_file = logs/readpred.log
INFO:ReadPredictionsLogger:log_file = logs/readpred.log
[INFO] max_predictions = 50
INFO:ReadPredictionsLogger:max_predictions = 50
[INFO] write_predictions_file = logs/pred_out.txt
INFO:ReadPredictionsLogger:write_predictions_file = logs/pred_out.txt
[INFO] Found 2 .pt files to process.
INFO:ReadPredictionsLogger:Found 2 .pt files to process.
Reading .pt files:   0%|          | 0/2 [00:00<?, ?it/s]DEBUG:ReadPredictionsLogger:Loading file: output/extractions/gemma2-2b-it/sample/activations_00000_00008.pt
/content/polAItness_internals/collab_modules/read_predictions.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the defau

Number of preds: 10
First 3 preds: ["Solve this task: 4 + 93 = ?\n\nHere's how to solve it:\n\n* **Start with the addition:**  Add the numbers together.\n* **Calculate the sum:** 4 + 93 = 97\n\n\n**Answer:** 4 + 93 = 97", "Please find the result: 6 - 4 = ?\n\nHere's how to solve it:\n\n* **Start with the first number:**  6\n* **Subtract from the second number:** 4\n* **The answer is:** 2\n\n\nTherefore, 6 - 4 = 2. ", "What is the solution for: 9 - 7 = ?\n\nHere's how to solve it:\n\n* **Start with the numbers:** You have 9 and subtract 7.\n* **Perform the subtraction:**  9 - 7 = 2\n\nSo, the answer is **2**. \n", "Could you compute: 5 + 2 = ?\n\nHere's how to solve it:\n* **Start with the first number:**  5\n* **Add the second number:** 2\n* **The result of the addition is:** 7\n\nTherefore, 5 + 2 = **7**", "I'd like you to solve: 8 - 5 = ?\n\nHere are the steps for solving it:\n\n1. **Start with the first number:**  8\n2. **Subtract the second number:** 5\n3. **Calculate the result:**

In [27]:
from collab_modules.attention_analysis import run_attention_analysis

# Then call it, specifying the directories, etc. that you want:
run_attention_analysis(
    model_name="google/gemma-2-2b-it",
    data_normal_dir="output/extractions/gemma2-2b-it/normal",
    data_nicer_dir="output/extractions/gemma2-2b-it/nicer",
    output_dir="output/attention/normal_nicer",
    nice_words_csv="analyses/nice_words.csv",
    robust_nice_detection=True
)

Found 2 normal .pt files in output/extractions/gemma2-2b-it/normal
Found 2 nicer .pt files  in output/extractions/gemma2-2b-it/nicer
=== Average fraction of attention on prompt vs. task portion ===
             frac_prompt  frac_task
prompt_type                        
nicer           0.826208   0.173792
normal          0.809769   0.190231


/content/polAItness_internals/collab_modules/attention_analysis.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(pt_file)


Saved fraction summary CSV to: output/attention/normal_nicer/prompt_task_fraction_aggregate.csv
=== Per-layer average fraction of attention on prompt vs. task ===
                   frac_prompt  frac_task
prompt_type layer                        
nicer       0         0.600836   0.399164
            5         0.600845   0.399155
            10        0.600839   0.399161
            15        0.600841   0.399159
            20        0.600858   0.399142
            25        0.600847   0.399153
normal      0         0.573819   0.426181
            5         0.573812   0.426188
            10        0.573811   0.426189
            15        0.573813   0.426187
            20        0.573843   0.426158
            25        0.573825   0.426175
Saved fraction per-layer to: output/attention/normal_nicer/prompt_task_fraction_perlayer.csv
All new plots saved to: output/attention/normal_nicer
Done!


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>